# Saliency Detection / Prediction.

## Tematem owej prezentacji jest image salience prediction / detection.

## Jest to problem związany z określeniem istotnych fragmentów zdjęcia. Istotność jest tu jednak pojęciem rozmytym.

Gdy taksówkarz kieruje pojazdem, istotne będą dla niego znaki drogowe, inne samochody, sygnalizacja, czy ogólnie sytuacja na drodze, a mniej reklamy na mijanych ścianach domów. Dla jego pasażera sytuacja może być całkiem odwrotna. Zatem określenie istotności dla podmiotu poznającego jest związane z akcją, którą podmiot ten aktualnie wykonuje, jego stanami wewnętrznymi. Taki pogląd na istotność obiektów zostanie omówiony w pierwszej kolejności. Takie podejście do silency detection używa się w Intepretable Machine Learning, gdzie bada się czy przetrenowany model dokonuje predykcji na podstawie cech istotnych dla klasy decyzyjnej, czy też wzorca właściwego dostępnym danym (na zdjęciu Husky występuje jedynie w śnieżnym krajobrazie, więc model przykłada równie istotną wagę do śniegu co do cech charakterystycznych dla Husky'ego).

Po co przydaje się saliency detection:
   > Interpretable Machine Learning: Wewnętrza ocena predykcji modelu
   > Object Detection/Segmentation: Wyodrębnienie ze zdjęcia istotnych obiektów
   > Wczesne przetworzenie danych: Wyodrębnienie istotniejszych części obrazu, pod zadanie właściwe

In [ ]:
Skoro 

## Saliency Maps

Saliency Maps to technika wizualizacji pozwalająca uzyskać lepszy wgląd w proces decyzyjny sieci neuronowej. Pomagają również wiedzieć, na czym skupia się każda warstwa warstwy splotowej. Pomaga nam to nieco lepiej zrozumieć proces podejmowania decyzji.

In [ ]:
import torch
import torchvision
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary
import requests
from PIL import Image

#Using VGG-19 pretrained model for image classification

model = torchvision.models.vgg19(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

In [ ]:
def download(url,fname):
    response = requests.get(url)
    with open(fname,"wb") as f:
        f.write(response.content)
    
# Downloading the image    
download("https://specials-images.forbesimg.com/imageserve/5db4c7b464b49a0007e9dfac/960x0.jpg?fit=scale","input.jpg")

# Opening the image
img = Image.open('input.jpg') 

In [ ]:
img

In [ ]:
# Preprocess the image
def preprocess(image, size=224):
    transform = T.Compose([
        T.Resize((size,size)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        T.Lambda(lambda x: x[None]),
    ])
    return transform(image)

'''
    Y = (X - μ)/(σ) => Y ~ Distribution(0,1) if X ~ Distribution(μ,σ)
    => Y/(1/σ) follows Distribution(0,σ)
    => (Y/(1/σ) - (-μ))/1 is actually X and hence follows Distribution(μ,σ)
'''
def deprocess(image):
    transform = T.Compose([
        T.Lambda(lambda x: x[0]),
        T.Normalize(mean=[0, 0, 0], std=[4.3668, 4.4643, 4.4444]),
        T.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
        T.ToPILImage(),
    ])
    return transform(image)

def show_img(PIL_IMG):
    plt.imshow(np.asarray(PIL_IMG))

In [ ]:
show_img(img)

In [ ]:
X = preprocess(img)

model.eval()

X.requires_grad_()

scores = model(X)

score_max_index = scores.argmax()
score_max = scores[0,score_max_index]

score_max.backward()

saliency, _ = torch.max(X.grad.data.abs(),dim=1)

plt.imshow(saliency[0], cmap=plt.cm.hot)
plt.axis('off')
plt.show()

In [ ]:
score_max

In [ ]:
X.grad.data.abs().shape

In [ ]:
saliency, _ = torch.max(X.grad.data.abs(),dim=1)

In [ ]:
saliency.shape

In [ ]:
X.grad